In [1]:
import pandas as pd

In [2]:
import sys
sys.path.insert(0,"../scoring")

In [3]:
from scoring_bhc import calculate_scores_bhc
from scoring_bhc import compute_overall_score_bhc

In [4]:
from scoring_instructions import calculate_scores_instructions
from scoring_instructions import compute_overall_score_instructions

# Test

In [5]:
# notes = pd.read_csv("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/comparison_simple_gpt3.5_2shot_v2.csv")
# notes = pd.read_csv("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/submission.csv")

In [6]:
notes = pd.read_pickle("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/bhc_scores/discharge_target_with_preceding_text+structured_data_BART_gen_test_phase_1_v3.pickle")

In [7]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [14]:
discharge_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge.csv.gz", keep_default_na=False)


In [15]:
notes = notes.merge(discharge_test[['hadm_id',"text"]], on="hadm_id")

In [38]:
notes

,Unnamed: 0,hadm_id,discharge_instruction_gpt,Original,text
0,0,26231944,You were admitted to the hospital for decompen...,"Dear Mr. ___,\n\nIt was a pleasure to care for...",\nName: ___ Unit No: ___\n ...
1,1,28542384,You were admitted to the hospital for a recent...,"Dear Ms. ___, \n We had the pleasure of taking...",\nName: ___ Unit No: ___\n \nA...
2,2,26489329,You were admitted to the hospital for a possib...,You were admitted to the hospital after a with...,\nName: ___ Unit No: ___\n ...
3,3,22641254,You were admitted to the hospital for hypoxemi...,"Dear Ms. ___, \n\nIt was a pleasure caring for...",\nName: ___ Unit No: ___\n ...
4,4,29265750,You were admitted to the hospital for right-si...,"Dear Mr. ___, \n\nYou were admitted to ___ wit...",\nName: ___ Unit No: ...
...,...,...,...,...,...
14692,14692,27445071,"Dear Mr. or Ms. [Patient's Last Name], You wer...","Dear Mr. ___,\n\nYou were admitted after getti...",\nName: ___ Unit No: ___\...
14693,14693,26192891,"Dear Mr. ___, You were admitted to the hospita...","Dear Mr. ___,\n\n You were admitted to ___ af...",\nName: ___ Unit No: ___\n ...
14694,14694,26584893,"Dear Ms. ___, You were admitted to the hospita...","Dear Ms. ___,\n\nIt was a pleasure taking ___ ...",\nName: ___ Unit No: _...
14695,14695,25623241,"Dear Mr. ___, You were admitted to the hospita...","Dear Mr. ___, \n\nYou were admitted to ___ \nb...",\nName: ___ Unit No: ___\n ...


In [39]:
# hadm_id,brief_hospital_course,discharge_instructions

In [9]:
notes[["brief_hospital_course_BART", 'brief_hospital_course']]

,brief_hospital_course_BART,brief_hospital_course
0,"___ with PMH HCV, ETOH cirrhosis with ascites,...","___ with PMH HCV, ETOH cirrhosis with ascites,..."
1,___ yo F s/p L4-5 discectomy/fusion (___) by \...,___ yo F with recent lumbar laminectomy c/b MS...
2,The patient was admitted to the colorectal \ns...,The patient was seen ___ the emergency departm...
3,"___ w/ h/o CAD (known 3VD) and ESRD on HD, \nw...","Ms. ___ is a ___ woman with a history of T2DM,..."
4,Mr. ___ is a ___ year old man with a history o...,___ with IDDM and h/o provoked PE on apixiban ...
...,...,...
14697,___ M PMHx dilated non-ischemic cardiomyopathy...,___ M PMHx dilated non-ischemic cardiomyopathy...
14698,___ year old M with PMH of CAD s/p 2xMI and co...,"Mr. ___ is a ___ year-old man with CAD, HTN, H..."
14699,"___ with CHF (EF 60%), a-fib on coumadin, \nHT...",___ year old female with past medical history ...
14700,This is a ___ with PMHx of hypertension and \n...,Mr. ___ is a ___ male with a past medical hist...


In [11]:
# reference = notes[['hadm_id', "Original"]].rename({"Original":"brief_hospital_course"}, axis=1)
# generated = notes[['hadm_id', "discharge_instruction_gpt"]].rename({"discharge_instruction_gpt":"brief_hospital_course"}, axis=1)

reference = notes[['hadm_id', "brief_hospital_course"]]
generated = notes[['hadm_id',
                   "brief_hospital_course_BART"]].rename({"brief_hospital_course_BART":"brief_hospital_course"}, axis=1)



In [12]:
notes.isna().sum(), generated.isna().sum()

(index                                0
 note_id                              0
 hadm_id                              0
 discharge_instructions               0
 brief_hospital_course                0
 discharge_instructions_word_count    0
 brief_hospital_course_word_count     0
 bhc_preceding_text                   0
 _merge                               0
 subject_id                           0
 note_type                            0
 note_seq                             0
 charttime                            0
 storetime                            0
 text                                 0
 age                                  0
 sex                                  0
 cc                                   0
 diags                                0
 procs                                0
 prescriptions                        0
 labs                                 0
 microbio                             0
 brief_hospital_course_BART           0
 dtype: int64,
 hadm_id                 

In [43]:
# looks like hadm_id 28358078 didn't get generated, and I think it throws an error, so we're dropping it for now
generated = generated[generated['hadm_id'] != 28358078]
reference = reference[reference['hadm_id'] != 28358078]


In [13]:
scores = calculate_scores_bhc(generated,
                 reference,
                 [
                     "bleu", 
                     "rouge", 
                     "bertscore", 
                     "meteor"
                 ])

Beginning scoring...
bleuScorer initialized
rougeScorer initialized
bertScorer initialized


[nltk_data] Downloading package wordnet to /home/vs428/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vs428/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vs428/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


meteorScorer initialized
Processed 128/14702 samples.
Processed 256/14702 samples.
Processed 384/14702 samples.
Processed 512/14702 samples.
Processed 640/14702 samples.
Processed 768/14702 samples.
Processed 896/14702 samples.
Processed 1024/14702 samples.
Processed 1152/14702 samples.
Processed 1280/14702 samples.
Processed 1408/14702 samples.
Processed 1536/14702 samples.
Processed 1664/14702 samples.
Processed 1792/14702 samples.
Processed 1920/14702 samples.
Processed 2048/14702 samples.
Processed 2176/14702 samples.
Processed 2304/14702 samples.
Processed 2432/14702 samples.
Processed 2560/14702 samples.
Processed 2688/14702 samples.
Processed 2816/14702 samples.
Processed 2944/14702 samples.
Processed 3072/14702 samples.
Processed 3200/14702 samples.
Processed 3328/14702 samples.
Processed 3456/14702 samples.
Processed 3584/14702 samples.
Processed 3712/14702 samples.
Processed 3840/14702 samples.
Processed 3968/14702 samples.
Processed 4096/14702 samples.
Processed 4224/14702 s

In [14]:
compute_overall_score_bhc(scores)

Computing overall score...
Done.


{'bleu': 0.0888808203249767,
 'rouge1': 0.37066980271149313,
 'rouge2': 0.16068241464160843,
 'rougel': 0.23320418250259925,
 'bertscore': 0.3758373872807007,
 'meteor': 0.27051062511516893,
 'overall': 0.24996420542942455}